In [ ]:
!pip3 install hdbscan

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from skimage import io
import multiprocessing
from joblib import Parallel, delayed
import wget
import os
import shutil
from google.cloud import storage
import sys



import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context('notebook', font_scale=1.5,
                rc={"lines.linewidth": 2.5})

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing.image import ImageDataGenerator
from keras import Model
from sklearn.utils import shuffle
import umap


from hdbscan import HDBSCAN

BUCKET_NAME = 'product-classification'
PROJECT_NAME = 'project_001_freisteller'
path = 'gs://' + BUCKET_NAME + '/' + PROJECT_NAME + '/'
IMAGE_PATH = 'images'
MOUNTED_PATH = '/home/jupyter/product-classification/'
CORE_FILE = 'OTTO_MAK_Bilddaten.xlsx'

Using TensorFlow backend.
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/py

In [ ]:
#core_df = pd.read_excel(path + CORE_FILE)

In [2]:
print((MOUNTED_PATH+PROJECT_NAME+'/'+IMAGE_PATH+'/'))
print(len([name for name in os.listdir((MOUNTED_PATH+PROJECT_NAME+'/'+IMAGE_PATH+'/'))]))

/home/jupyter/product-classification/project_001_freisteller/images/
959


In [ ]:
#!mkdir /home/jupyter/product-classification

In [ ]:
!/usr/bin/gcsfuse product-classification /home/jupyter/product-classification

In [ ]:
#!ls /home/jupyter/product-classification/

*Hier fängt das Trainieren des InceptionResNetV2-Modells an*

Beschreibung einzelner Prozessschritte des Gesamtprozesses:
1. Basismodell von InceptionResNetV2 anwenden. Wir verwenden die vortrainierten Gewichte anhand Imagenet, werden später eigene Output-Schichten (z.B MaxPooling layer, Dense layer mit bestimmter Anzahl von Klassen) einbauen.
=> include_top= False
2. ImageDataGenerator werden verwendet, um Pixeln von Bildern zu kodieren.
3. wir definieren Trainingsdaten, Validierungsdaten und Testdaten
4. Wir verwenden flow_from_dataframe, um Bilder aus dem Ordner zu lesen und sie jeweils zu ihrer Klasse zuzuordnen. Dafür ist eine Tabelle mit zwei Spalten (filenames und classes) nötig. 
5. Für jede Tabelle (Trainings-,Validierung-,Testdaten) flow_from_dataframe anwenden. Bei Trainingsdaten und Validierungsdaten: Shuffel: True, y_col: Klasse, Bei Testdaten: Shuffel: false, y_col: None, default Batch_Size= 1
6. Batch_Size so definieren, dass sie nicht größer als die Datenmenge ist.
7. Basismodell mit zusätzlichen Schichten einbauen. Somit entsteht das endgültige Modell.
8. Mit fit_generator das Modell mit den Trainingsdaten trainieren.
9. Mit predict_generator das Modell mit den Testdaten testen. 
10. Mit predicted_class_indices bekommen wir am Ende vom Modell vorhergesagte Klassen für jeweilige Datensätze in den Testdaten. 




In [3]:
# prepare model to encode images

incesnet = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299,299,3))

generator = ImageDataGenerator(rescale= 1. / 255)
generator2= ImageDataGenerator(rescale= 1. / 255)

In [4]:
classified_df = pd.read_csv(path + 'train_classifier.csv')
classified_df['Label'] = classified_df['Label'].astype(str)


In [5]:
label_names = {0: 'Zweifarbig',
               1: 'Einfarbig',
               2: 'Freisteller',
               3: 'Ambiente',
               4: 'Abmaßungen',
               5: 'Sonstige'}

In [6]:
BATCH_SIZE = 35
# set the path to the serialized model after training
MODEL_PATH = os.path.sep.join(["output", "food11.model"])
 
# define the path to the output training history plots
UNFROZEN_PLOT_PATH = os.path.sep.join(["output", "unfrozen.png"])
WARMUP_PLOT_PATH = os.path.sep.join(["output", "warmup.png"])

In [7]:
#trainingsdata =classified_df[:100]
#validationdata=classified_df[100:200]
#testdata=classified_df[200:300]

In [8]:
def plot_training(H, N, plotPath):
    # construct a plot that plots and saves the training history
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
    plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig(plotPath)

In [9]:
trainingsdata = classified_df.sample(n=300)
validationdata = classified_df.sample(n=50)
testdata = classified_df.sample(n=50)

In [10]:
trainingsdata_overview = trainingsdata.groupby('Label', sort=False).size().reset_index(name='Count')
print (trainingsdata_overview)

  Label  Count
0     5     30
1     2     91
2     3     69
3     1     38
4     4     48
5     0     24


In [ ]:
training_gen = generator.flow_from_dataframe(dataframe=trainingsdata,
                                    shuffle=True,
                                    directory=(MOUNTED_PATH + PROJECT_NAME + '/' + IMAGE_PATH),
                                    x_col='Image',
                                    y_col='Label',
                                    target_size=(299, 299),
                                    batch_size=BATCH_SIZE,  
                                    class_mode='categorical')

In [ ]:
valid_gen = generator2.flow_from_dataframe(dataframe=validationdata,
                                    shuffle=True,
                                    directory=(MOUNTED_PATH + PROJECT_NAME + '/' + IMAGE_PATH),
                                    x_col='Image',
                                    y_col='Label',
                                    target_size=(299, 299),
                                    batch_size=BATCH_SIZE,  
                                    class_mode='categorical')

In [ ]:
test_gen = generator2.flow_from_dataframe(dataframe=testdata,
                                    shuffle=False,
                                    directory=(MOUNTED_PATH + PROJECT_NAME + '/' + IMAGE_PATH),
                                    x_col='Image',
                                    y_col=None ,
                                    target_size=(299, 299),
                                    batch_size=1,  
                                    class_mode=None)

In [ ]:
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Input, Dense, BatchNormalization, Dropout, Activation, PReLU
from keras.layers.core import Dropout
from keras.layers.core import Flatten

x = incesnet.output
x = Flatten(name="flatten")(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
#x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='relu')(x)
predictions = Dense(len(label_names), activation='softmax')(x) 
model = Model(inputs=incesnet.input, outputs=predictions)
# Freezing Convolutional layers
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in incesnet.layers:
    layer.trainable = False
    
print(model.summary())
        
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=training_gen.n//training_gen.batch_size
STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size
STEP_SIZE_TEST=test_gen.n//test_gen.batch_size

In [ ]:
STEP_SIZE_TRAIN

In [ ]:
H = model.fit_generator(generator=training_gen,
                     steps_per_epoch = STEP_SIZE_TRAIN,
                      validation_data=valid_gen, #valid_gen
                       validation_steps=STEP_SIZE_VALID,
                         epochs = 20)

In [ ]:
test_gen.reset()
pred=model.predict_generator(test_gen,
steps=STEP_SIZE_TEST,
verbose=1)

In [ ]:
from sklearn.metrics import classification_report
#from pyimagesearch import config

predicted_class_indices=np.argmax(pred,axis=1)
labels = (training_gen.class_indices)
labels2 = dict((v,k) for k,v in label_names.items())
predictions = [label_names[k] for k in predicted_class_indices]
print(predicted_class_indices)
print (labels)
print (predictions)


In [ ]:
predictions2 = [k for k in predicted_class_indices]

In [ ]:
target_names = ['Zweifarbig',
                'Einfarbig',
                'Freisteller',
                'Ambiente',
                'Abmaßungen',
                'Sonstige']

print(classification_report(testdata['Label'].astype(str).astype(int),predictions2, target_names = target_names))

In [ ]:
def plots(hist):
    """
    Plots the plots from a training history.
    Parameters:
    -----------
    history : history of a model fit (required)
    """
    # summarize history for accuracy
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.clf()

    # summarize history for loss
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid('off')
    plt.tight_layout()

In [ ]:
plots(H)

In [ ]:
# reset our data generators
training_gen.reset()
valid_gen.reset()
 
# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in incesnet.layers[15:]:
    layer.trainable = True

for layer in incesnet.layers:
    print("{}: {}".format(layer, layer.trainable))

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD

# for the changes to the model to take affect we need to recompile
opt = SGD(lr=1e-4, momentum=0.9)

print("[INFO] re-compiling model...")
model.compile(loss="categorical_crossentropy", optimizer = opt, 
    metrics=["accuracy"])
 
# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
H = model.fit_generator(
    training_gen,
    steps_per_epoch = STEP_SIZE_TRAIN,
    validation_data=valid_gen, #valid_gen
    validation_steps=STEP_SIZE_VALID,
    epochs=10)

In [ ]:
# reset the testing generator and then use our trained model to
# make predictions on the data

print("[INFO] evaluating after fine-tuning network...")
test_gen.reset()
pred = model.predict_generator(test_gen,
    steps=STEP_SIZE_TEST,
    verbose=1)


In [ ]:
from sklearn.metrics import classification_report


predicted_class_indices=np.argmax(pred,axis=1)
labels = (training_gen.class_indices)
labels2 = dict((v,k) for k,v in label_names.items())
predictions = [label_names[k] for k in predicted_class_indices]
print(predicted_class_indices)
print (labels)
print (predictions)

predictions2 = [k for k in predicted_class_indices]

target_names = ['Zweifarbig',
                'Einfarbig',
                'Freisteller',
                'Ambiente',
                'Abmaßungen',
                'Sonstige']

print(classification_report(testdata['Label'].astype(str).astype(int),predictions2, target_names = target_names))



#plot_training(H, 10, UNFROZEN_PLOT_PATH)
 
# serialize the model to disk
#print("[INFO] serializing network...")
#model.save(config.MODEL_PATH)

In [ ]:
########## test sample pictures ############

testdata2 = classified_df.sample(n=1)

print(testdata2)

sample_gen = generator2.flow_from_dataframe(dataframe=testdata2,
                                    shuffle=False,
                                    directory=(MOUNTED_PATH + PROJECT_NAME + '/' + IMAGE_PATH),
                                    x_col='Image',
                                    y_col=None ,
                                    target_size=(299, 299),
                                    batch_size=1,  
                                    class_mode=None)

#sample_gen.reset()
pred = model.predict_generator(sample_gen,
    verbose=1)


predicted_class_indices=np.argmax(pred,axis=1)
labels = (training_gen.class_indices)
predictions = [label_names[k] for k in predicted_class_indices]


print("prediction: ")
print(predicted_class_indices)
print (predictions)

In [ ]:
from PIL import Image, ImageDraw

image = Image.open('/home/jupyter/product-classification/project_001_freisteller/images/20192535.jpg') #jpg
image.show()
image

In [ ]:
#save result

#imagefile=test_gen.filenames
#results=pd.DataFrame({"Image":imagefile,
#                      "Prediction":predictions})
#results.to_csv("results_test1.csv",index=False)

In [ ]:
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
import datetime

# Export the model to a file
filename = 'model.joblib'

joblib.dump(model, filename)
 


# Upload the model to GCS
bucket = storage.Client().bucket(BUCKET_NAME)
blob = bucket.blob('{}/{}'.format(
    datetime.datetime.now().strftime('census_%Y%m%d_%H%M%S'),
    model))
blob.upload_from_filename(model)

--------------------------------------------------------------------------

Bis Hier :) 

In [ ]:
from google.cloud import storage

BUCKET_NAME = 'product-classification'
PROJECT_NAME = 'project_001_freisteller'


storage_client = storage.Client()

public_bucket = storage_client.bucket(BUCKET_NAME)
blob = public_bucket.blob('project_001_freisteller/classified.csv')
blob.download_to_filename('classified.csv')

In [ ]:
import time

# Define a timestamped job name
JOB_NAME = "census_training_{}".format(int(time.time()))

In [ ]:
!touch ./__init__.py

In [ ]:
# Submit the training job:
!gcloud ai-platform jobs submit training $JOB_NAME \
  --job-dir gs://$BUCKET_NAME/project_001_freisteller \
  --package-path ./ \
  --module-name project_001_freisteller.train \
  --region us-central1 \
  --runtime-version=1.12 \
  --python-version=3.5 \
  --scale-tier BASIC 

In [ ]:
!gcloud ai-platform jobs submit training --help